https://towardsdatascience.com/understanding-actor-critic-methods-931b97b6df3f

In [ ]:
!pip install yfinance
!pip install yahoofinancials

In [ ]:
import numpy as np
import pandas as pd
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from IPython.utils.text import long_substr

from collections import deque
import random

import yfinance as yf
from yahoofinancials import YahooFinancials

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataGoogle = yf.download("GOOG", start="2016-01-01", end="2017-12-31")
dataGoogle['Date'] = dataGoogle.index
dataGoogle = dataGoogle[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
dataGoogle = dataGoogle.reset_index(drop=True)
dataGoogle['Date'] = pd.to_datetime(dataGoogle['Date']).apply(lambda x: x.date())
dataGoogle_train = dataGoogle
dataGoogle_train

In [ ]:
dataGoogle2 = yf.download("GOOG", start="2018-01-01", end="2018-05-31")
dataGoogle2['Date'] = dataGoogle2.index
dataGoogle2 = dataGoogle2[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
dataGoogle2 = dataGoogle2.reset_index(drop=True)
dataGoogle2['Date'] = pd.to_datetime(dataGoogle2['Date']).apply(lambda x: x.date())
dataGoogle_test = dataGoogle2
dataGoogle_test

In [ ]:
dataGoogle_train.to_csv('/content/drive/MyDrive/Fall 2022/Reinforcement Learning/Project/dataGoogle_train_2y.csv')
dataGoogle_test.to_csv('/content/drive/MyDrive/Fall 2022/Reinforcement Learning/Project/dataGoogle_test_1y.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Fall 2022/Reinforcement Learning/Project/dataGoogle_train_AC.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Fall 2022/Reinforcement Learning/Project/dataGoogle_test_1y.csv')
df_test = df_test.drop(['Unnamed: 0'], axis=1)
df_test.head()

#Agent

In [ ]:
class Actor:
    def __init__(self, name, input_size, output_size, size_layer):
        with tf.variable_scope(name):
            self.X = tf.placeholder(tf.float32, (None, None, input_size)) #Input
            self.hidden_layer = tf.placeholder(tf.float32, (None, 2 * size_layer)) #Middle_layer
            cell = tf.nn.rnn_cell.LSTMCell(size_layer, state_is_tuple = False) #LSTM
            self.rnn,self.last_state = tf.nn.dynamic_rnn(inputs=self.X, cell=cell, dtype=tf.float32, initial_state=self.hidden_layer) #RNN
            self.logits = tf.layers.dense(self.rnn[:,-1], output_size) #Output_Forward

In [ ]:
class Critic:
    def __init__(self, name, input_size, output_size, size_layer, learning_rate):
        with tf.variable_scope(name):
            self.X = tf.placeholder(tf.float32, (None, None, input_size)) #Input
            self.Y = tf.placeholder(tf.float32, (None, output_size)) #Output
            self.hidden_layer = tf.placeholder(tf.float32, (None, 2 * size_layer)) #Hidden_layer
            self.REWARD = tf.placeholder(tf.float32, (None, 1)) #Reward
            feed_critic = tf.layers.dense(self.X, size_layer, activation = tf.nn.relu) #Middle_layer1
            cell = tf.nn.rnn_cell.LSTMCell(size_layer, state_is_tuple = False) #LSTM
            self.rnn,self.last_state = tf.nn.dynamic_rnn(inputs=self.X, cell=cell, dtype=tf.float32, initial_state=self.hidden_layer) #RNN
            feed_critic = tf.layers.dense(self.rnn[:,-1], output_size, activation = tf.nn.relu) + self.Y #Middle_layer2
            feed_critic = tf.layers.dense(feed_critic, size_layer//2, activation = tf.nn.relu) #Middle_layer3
            self.logits = tf.layers.dense(feed_critic, 1) #Output_Forward
            self.loss = tf.reduce_mean(tf.square(self.REWARD - self.logits)) #Average_loss
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.loss) #Optimizer

In [ ]:
class Agent:

    LEARNING_RATE = 0.001
    BATCH_SIZE = 32
    LAYER_SIZE = 256
    OUTPUT_SIZE = 3
    EPSILON = 0.5
    DECAY_RATE = 0.005
    MIN_EPSILON = 0.1
    GAMMA = 0.99
    MEMORIES = deque()
    MEMORY_SIZE = 300
    COPY = 1000
    T_COPY = 0

    def __init__(self, state_size, window_size, trend, test_data):
        self.state_size = state_size
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend
        self.test_data = test_data
        self.INITIAL_FEATURES = np.zeros((4, self.state_size))

      # Neural Network Model
        tf.reset_default_graph()
        self.actor = Actor('actor-original', self.state_size, self.OUTPUT_SIZE, self.LAYER_SIZE)
        self.actor_target = Actor('actor-target', self.state_size, self.OUTPUT_SIZE, self.LAYER_SIZE)
        self.critic = Critic('critic-original', self.state_size, self.OUTPUT_SIZE, self.LAYER_SIZE, self.LEARNING_RATE)
        self.critic_target = Critic('critic-target', self.state_size, self.OUTPUT_SIZE, self.LAYER_SIZE, self.LEARNING_RATE)
        self.grad_critic = tf.gradients(self.critic.logits, self.critic.Y)
        self.actor_critic_grad = tf.placeholder(tf.float32, [None, self.OUTPUT_SIZE])
        weights_actor = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='actor')
        self.grad_actor = tf.gradients(self.actor.logits, weights_actor, -self.actor_critic_grad)
        grads = zip(self.grad_actor, weights_actor)
        self.optimizer = tf.train.AdamOptimizer(self.LEARNING_RATE).apply_gradients(grads)
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
    
    def _assign(self, from_name, to_name):
        from_w = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=from_name)
        to_w = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=to_name)
        for i in range(len(from_w)):
            assign_op = to_w[i].assign(from_w[i])
            self.sess.run(assign_op)
            
    def _memorize(self, state, action, reward, new_state, done, rnn_state):
        self.MEMORIES.append((state, action, reward, new_state, done, rnn_state))
        if len(self.MEMORIES) > self.MEMORY_SIZE:
            self.MEMORIES.popleft()

    def act(self, state):
        if np.random.rand() < self.EPSILON:
            action = np.random.randint(self.OUTPUT_SIZE) #Random
        else:
            prediction = self.sess.run(self.actor.logits, feed_dict={self.actor.X:[state]})[0]
            action = np.argmax(prediction) #Greedy
        return action
    
    def replay(self, replay1):
        states = np.array([a[0] for a in replay1])
        new_states = np.array([a[3] for a in replay1])
        init_values = np.array([a[-1] for a in replay1])
        Q = self.sess.run(self.actor.logits, feed_dict={self.actor.X: states, self.actor.hidden_layer: init_values})
        Q_target = self.sess.run(self.actor_target.logits, feed_dict={self.actor_target.X: states, self.actor_target.hidden_layer: init_values})
        grads = self.sess.run(self.grad_critic, feed_dict={self.critic.X:states, self.critic.Y:Q, self.critic.hidden_layer: init_values})[0]
        self.sess.run(self.optimizer, feed_dict={self.actor.X:states, self.actor_critic_grad:grads, self.actor.hidden_layer: init_values})
        
        rewards = np.array([a[2] for a in replay1]).reshape((-1, 1))
        rewards_target = self.sess.run(self.critic_target.logits, feed_dict={self.critic_target.X:new_states,self.critic_target.Y:Q_target, self.critic_target.hidden_layer: init_values})
        
        for i in range(len(replay1)):
            if not replay1[0][-2]:
                rewards[i] += self.GAMMA * rewards_target[i]
        loss, _ = self.sess.run([self.critic.loss, self.critic.optimizer], 
                                feed_dict={self.critic.X:states, self.critic.Y:Q, self.critic.REWARD:rewards,
                                          self.critic.hidden_layer: init_values})
        return loss
    
    def get_state(self, t):
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 1):
            res.append(block[i + 1] - block[i])
        return np.array(res)


    def train(self, iterations, checkpoint, initial_money):
        lossV = []
        profitV = []
        profitIter = []
        for i in range(iterations):
            loss_temp = []
            profit_temp = []
            total_profit = 0
            inventory = []
            state = self.get_state(0)
            starting_money = initial_money
            init_value = np.zeros((1, 2 * self.LAYER_SIZE))
            for k in range(self.INITIAL_FEATURES.shape[0]):
                self.INITIAL_FEATURES[k,:] = state
            for t in range(0, len(self.trend) - 1):
                if (self.T_COPY + 1) % self.COPY == 0:
                    self._assign('actor-original', 'actor-target')
                    self._assign('critic-original', 'critic-target')
                    
                if np.random.rand() < self.EPSILON:
                    action = np.random.randint(self.OUTPUT_SIZE)
                else:
                    action, last_state = self.sess.run([self.actor.logits,
                                                  self.actor.last_state],
                                                  feed_dict={self.actor.X:[self.INITIAL_FEATURES],
                                                             self.actor.hidden_layer:init_value})
                    action, init_value = np.argmax(action[0]), last_state
                
                next_state = self.get_state(t + 1)
                
              #Buy
                if action == 1 and starting_money >= self.trend[t]:
                    inventory.append(self.trend[t])
                    starting_money -= self.trend[t]
                
              #Sell
                elif action == 2 and len(inventory) > 0:
                    bought_price = inventory.pop(0)
                    total_profit += self.trend[t] - bought_price
                    starting_money += self.trend[t]
                    
              #Reward
                reward = ((starting_money - initial_money) / initial_money)
                new_state = np.append([self.get_state(t + 1)], self.INITIAL_FEATURES[:3, :], axis = 0)

                self._memorize(self.INITIAL_FEATURES, action, reward, new_state, 
                               starting_money < initial_money, init_value[0])
                batch_size = min(len(self.MEMORIES), self.BATCH_SIZE)
                self.INITIAL_FEATURES = new_state
                replay1 = random.sample(self.MEMORIES, batch_size)
                loss = self.replay(replay1)
                loss_temp.append(loss)
                profit_temp.append(total_profit)
                self.T_COPY += 1
                self.EPSILON = self.MIN_EPSILON + (1.0 - self.MIN_EPSILON) * np.exp(-self.DECAY_RATE * i)
            if (i+1) % checkpoint == 0:
                print('epoch: %d, total profit: %f.3, loss: %f, total money: %f'%(i + 1, total_profit, loss, starting_money))
                profitIter.append(total_profit)
            lossV.append(np.mean(loss_temp))
            profitV.append(np.mean(profit_temp))
        return lossV, profitV, profitIter


    def evaluate(self, initial_money):
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        state = self.get_state(0)
        init_value = np.zeros((1, 2 * self.LAYER_SIZE))
        for k in range(self.INITIAL_FEATURES.shape[0]):
            self.INITIAL_FEATURES[k,:] = state
        for t in range(0, len(self.test_data) - 1):
            
            if np.random.rand() < self.EPSILON:
                action = np.random.randint(self.OUTPUT_SIZE)
            else:
                action, last_state = self.sess.run([self.actor.logits,
                                                  self.actor.last_state],
                                                  feed_dict={self.actor.X:[self.INITIAL_FEATURES],
                                                             self.actor.hidden_layer:init_value})
                action, init_value = np.argmax(action[0]), last_state
                    
            next_state = self.get_state(t + 1)
            
          #Buy
            if action == 1 and initial_money >= self.test_data[t]:
                inventory.append(self.test_data[t])
                initial_money -= self.test_data[t]
                states_buy.append(t)
                # print('day %d: buy 1 unit at price %f, total balance %f'% (t, self.test_data[t], initial_money))
            
          #Sell
            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                initial_money += self.test_data[t]
                states_sell.append(t)
                try:
                    reward = ((close[t] - bought_price) / bought_price) * 100
                except:
                    reward = 0
                # print('day %d, sell 1 unit at price %f, investment %f %%, total balance %f,' % (t, close[t], reward, initial_money))
            
            new_state = np.append([self.get_state(t + 1)], self.INITIAL_FEATURES[:3, :], axis = 0)
            self.INITIAL_FEATURES = new_state
        reward = ((initial_money - starting_money) / starting_money) * 100
        total_gains = initial_money - starting_money
        return states_buy, states_sell, total_gains, reward

In [ ]:
close = df.Close.values.tolist()
close_test = df_test.Close.values.tolist()
initial_money = 10000
window_size = 5
agent = Agent(state_size = window_size, 
              window_size = window_size, 
              trend = close, 
              test_data = close_test)
lossV, profitV, profitIter = agent.train(iterations = 200, checkpoint = 10, initial_money = initial_money)

In [ ]:
fig = plt.figure(figsize = (20,10))
plt.plot(lossV, color='r')
plt.title('Loss during Training')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.show()

In [ ]:
fig = plt.figure(figsize = (20,10))
plt.plot(profitV, color='r')
plt.title('Profit during Training (Final Profit: %f)'%(profitV[len(profitV)-1]))
plt.xlabel('Iterations')
plt.ylabel('Profit')
plt.show()

In [ ]:
fig = plt.figure(figsize = (20,10))
plt.plot(profitIter, color='r')
plt.title('Profit during Training (Final Profit: %f)'%(profitIter[len(profitIter)-1]))
plt.xlabel('Iterations')
plt.ylabel('Profit')
plt.show()

#Test

In [ ]:
states_buy, states_sell, total_gains, reward = agent.evaluate(initial_money = initial_money)

In [ ]:
fig = plt.figure(figsize = (20,10))
plt.plot(close_test, color='r', lw=2.)
plt.plot(close_test, '^', markersize=10, color='m', label = 'buying signal', markevery = states_buy)
plt.plot(close_test, 'v', markersize=10, color='k', label = 'selling signal', markevery = states_sell)
plt.title('Testing Close Price (Total Profit: %f)'%(total_gains))
plt.legend()
plt.show()